In [ ]:
%matplotlib inline
from scipy import integrate
import numpy as np
import time
from parex import parex
import matplotlib.pyplot as plt
from problems import nbody, kdv, burgers
import os.path

In [ ]:
tol = 1.e-11
problem = burgers
solver = 'explicit midpoint'
#solver = 'scipy DOP853'
if 'scipy' in solver:
    solver_name = solver.split()[-1].lower()
    r = integrate.complex_ode(problem.rhs_reversed).set_integrator(solver_name, atol=tol, rtol=tol, 
                                                                   verbosity=10, nsteps=1e7)
    t0 = problem.output_times[0]
    problem.output_times[-1]
    t0 = problem.output_times[0]
    tf = problem.output_times[-1]
    r.set_initial_value(problem.y0, t0)
    r.integrate(r.t+(tf-t0))
    assert r.t == tf, "Integration failed. Try increasing the maximum allowed steps."
    y = r.y
else:
    y, diagnostics = parex.solve(problem.rhs, problem.output_times,
                                 problem.y0, solver=solver, atol=tol,
                                 rtol=tol, diagnostics=True,
                                 jac_fun=problem.jacobian,
                                 max_steps=1.e8)
    y = np.squeeze(y[-1,:])

if problem.solout: y = problem.solout(y)


plt.plot(y);

In [ ]:
np.savetxt('./reference_data/' + problem.name + ".txt", y)

In [ ]:
def generate_reference_data(problem,use_scipy=False,reftol=1.e-14):
    "Run problem and save output for future reference.  Non-dense output."
    print("Generating reference data for " + problem.name)
    tol = 1.e-14
    if use_scipy:
        r = integrate.complex_ode(problem.rhs_reversed).set_integrator('DOP853', atol=tol, rtol=tol, 
                                                                       verbosity=10, nsteps=1e7)
        t0 = problem.output_times[0]
        problem.output_times[-1]
        r.set_initial_value(problem.y0, t0)
        r.integrate(r.t+(tf-t0))
        assert r.t == tf, "Integration failed. Try increasing the maximum allowed steps."
        y = r.y
    else:
        y, diagnostics = parex.solve(problem.rhs, problem.output_times,
                             problem.y0, solver=solver, atol=tol,
                             rtol=tol, diagnostics=True,
                             jac_fun=problem.jacobian,
                             max_steps=1e8)
        y = y[-1,:].squeeze()

    if problem.solout:
        y = problem.solout(y)
    np.savetxt('./reference_data/' + problem.name + ".txt", y)

In [ ]:
solvers = ['explicit euler', 'explicit midpoint','scipy DOPRI5', 'scipy DOP853']
tols = [1.e-3, 1.e-5, 1.e-7, 1.e-9]#, 1.e-11, 1.e-13]
problems = [nbody, kdv, burgers]

results = {}

# Check that all required reference data is available
for problem in problems:
    reference_file = "./reference_data/" + problem.name + ".txt"
    if not os.path.exists(reference_file):
        generate_reference_data(problem)
    
for problem in problems:
    results[problem.name] = {}
    reference_file = "./reference_data/" + problem.name + ".txt"
    y_ref = np.loadtxt(reference_file)
    for solver in solvers:
        results[problem.name][solver] = {}
        for tol in tols:
            print(problem.name, solver, tol)
            results[problem.name][solver][tol] = {}
            start = time.time()
            if 'scipy' in solver:
                solver_name = solver.split()[-1].lower()
                if problem in [kdv, burgers]:
                    r = integrate.complex_ode(problem.rhs_reversed).set_integrator(solver_name, atol=tol, 
                                                                          rtol=tol, verbosity=10, 
                                                                          nsteps=1e6)
                else:
                    r = integrate.ode(problem.rhs_reversed).set_integrator(solver_name, atol=tol, rtol=tol, 
                                                                  verbosity=10, nsteps=1e7)
                t0 = problem.output_times[0]
                tf = problem.output_times[-1]
                r.set_initial_value(problem.y0, t0)
                r.integrate(r.t+(tf-t0))
                assert r.t == tf, "Integration failed. Try increasing the maximum allowed steps."
                y = r.y


                results[problem.name][solver][tol]['fe_seq'] = None
                results[problem.name][solver][tol]['mean_order'] = None
            else:
                y, diagnostics = parex.solve(problem.rhs, problem.output_times,
                                             problem.y0, solver=solver, atol=tol,
                                             rtol=tol, diagnostics=True,
                                             jac_fun=problem.jacobian)

                results[problem.name][solver][tol]['mean_order'] = diagnostics["k_avg"]
                results[problem.name][solver][tol]['fe_seq'] = diagnostics["fe_seq"]
                y = y[-1,:].squeeze()

            end = time.time()
            if problem.solout:
                y = problem.solout(y)
            results[problem.name][solver][tol]['wall_time'] = end-start

            results[problem.name][solver][tol]['fe_total'] = np.sum(diagnostics["nfe"])
            results[problem.name][solver][tol]['nsteps'] = np.sum(diagnostics["nst"])
            results[problem.name][solver][tol]['je_total'] = np.sum(diagnostics["nje"])

            #plt.plot(problem.output_times,y)
            #plt.title(str(tol)+' '+solver)

            y = y.reshape(-1)
            y_ref = y_ref.reshape(-1)

            results[problem.name][solver][tol]['relative_error'] = np.linalg.norm(y-y_ref)/np.linalg.norm(y_ref)
            print(results[problem.name][solver][tol]['relative_error'])
            print(results[problem.name][solver][tol]['wall_time'])

In [ ]:
problem = kdv
for solver in solvers:
    wall_time = [results[problem.name][solver][tol]['wall_time'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, wall_time, '-s')
plt.legend(solvers);
plt.title(problem.name)
plt.savefig(problem.name+'.pdf')

In [ ]:
problem = burgers
for solver in solvers:
    wall_time = [results[problem.name][solver][tol]['wall_time'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, wall_time, '-s')
plt.legend(solvers);
plt.title(problem.name)
plt.savefig(problem.name+'.pdf')

In [ ]:
problem = nbody
for solver in solvers:
    wall_time = [results[problem.name][solver][tol]['wall_time'] for tol in tols]
    errors = [results[problem.name][solver][tol]['relative_error'] for tol in tols]
    plt.loglog(errors, wall_time, '-s')
plt.legend(solvers);
plt.title(problem.name)
plt.savefig(problem.name+'.pdf')

In [ ]:
kdv.solout

In [ ]:
plt.savefig?

In [ ]:
results['kdv']['explicit midpoint'][1.e-7]['relative_error']

In [ ]:
results['kdv']['scipy DOPRI5'][1.e-7]['relative_error']

In [ ]:
results['nbody']['scipy DOPRI5'][1.e-7]['wall_time']

In [ ]:
results['nbody']['explicit midpoint'][1.e-7]['relative_error']

In [ ]:
results['nbody']['scipy DOPRI5'][1.e-7]['relative_error']